# Task 1

Install Libraries

In [ ]:
!pip install --quiet --upgrade google-cloud-logging google_cloud_firestore google_cloud_aiplatform langchain langchain-google-vertexai langchain_community langchain_experimental pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 368.8/368.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00


In [ ]:
import vertexai
import logging
import google.cloud.logging
from vertexai.language_models import TextEmbeddingModel
from vertexai.generative_models import GenerativeModel

import pickle
from IPython.display import display, Markdown

from langchain_google_vertexai import VertexAIEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_experimental.text_splitter import SemanticChunker

from google.cloud import firestore
from google.cloud.firestore_v1.vector import Vector
from google.cloud.firestore_v1.base_vector_query import DistanceMeasure

Set Variables and VertaxAI.init()

In [ ]:
PROJECT_ID="qwiklabs-gcp-01-4db5c0a69bf4"
LOCATION="us-central1"
vertexai.init(project=PROJECT_ID, location=LOCATION)

Set embedding Model

In [ ]:
embedding_model = VertexAIEmbeddings(model_name="text-embedding-005")

# Task 2 Download, process and chunk data semantically


In this section, you will prepare the NYC Food Safety Manual for Retrieval-Augmented Generation (RAG). Clean the PDF content and split it into meaningful chunks based on semantic similarity using sentence embeddings and generate numerical representations (embeddings) for each identified text chunk.

Download the New York City Department of Health and Mental Hygiene's Food Protection Training Manual. This document will serve as your Retrieval-Augmented Generation source content.

In [ ]:
!gcloud storage cp gs://partner-genai-bucket/genai069/nyc_food_safety_manual.pdf .

Copying gs://partner-genai-bucket/genai069/nyc_food_safety_manual.pdf to file://./nyc_food_safety_manual.pdf

Average throughput: 112.9MiB/s


Use the LangChain class [PyMuPDFLoader](https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/#using-pymupdf) to load the contents of the PDF to a variable named data.

The following function is provided to do some basic cleaning on artifacts found in this particular document. Create a variable called cleaned_pages that is a list of strings, with each string being a page of content cleaned by this function.

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("nyc_food_safety_manual.pdf")
data = loader.load()

In [ ]:
def clean_page(page):
  return page.page_content.replace("-\n","")\
                          .replace("\n"," ")\
                          .replace("\x02","")\
                          .replace("\x03","")\
                          .replace("fo d P R O T E C T I O N  T R A I N I N G  M A N U A L","")\
                          .replace("N E W  Y O R K  C I T Y  D E P A R T M E N T  O F  H E A L T H  &  M E N T A L  H Y G I E N E","")

# Clean pages into list of strings
cleaned_pages = [clean_page(page) for page in data]

Use LangChain's [SemanticChunker](https://python.langchain.com/v0.2/docs/how_to/semantic-chunker/#create-text-splitter) with the embedding_model you created earlier to split the first five pages of cleaned_pages into text chunks. The SemanticChunker determines when to start a new chunk when it encounters a larger distance between sentence embeddings. Save the strings of page content from the resulting documents into a list of strings called chunked_content. Take a look at a few of the chunks to get familiar with the content.

Use the embedding_model to generate embeddings of the text chunks, saving them to a list called chunked_embeddings. To do so, pass your list of chunks to the [VertexAIEmbeddings](https://python.langchain.com/docs/integrations/text_embedding/google_vertex_ai_palm/) class's embed_documents() method.

You should have successfully chunked & embedded a short section of the document. To get the chunks & corresponding embeddings for the full document, run the following code:

In [ ]:
from langchain_experimental.text_splitter import SemanticChunker

# Use embedding_model from Task 1
splitter = SemanticChunker(embedding_model)

# Chunk the first 5 cleaned pages
docs = splitter.create_documents(cleaned_pages[:5])

# Extract only the chunk text into a list
chunked_content = [doc.page_content for doc in docs]

# Preview
for chunk in chunked_content[:3]:
    print("🧩 Chunk:\n", chunk[:300], "\n---")

🧩 Chunk:
 The Health Code These are regulations that were formulated to allow the  Department to effectively protect the health of the population. Among the rules embodied in the Health Code is Article 81 which regulates the operations of food establishments for the purpose of preventing public health hazards 
---
🧩 Chunk:
 Registration is done on-line. The link is: nyc.gov/foodprotectioncourse Register for Health Academy Classes On-Line You may now register and pay online for courses offered at the Department of Health and Mental Hygiene’s Health Academy, including the Food Protection Course for restaurants. This new  
---
🧩 Chunk:
 If you don’t see a date that is convenient, check back as new course dates are added frequently. 1   INTRODUCTION T he New York City Department of Health and Mental Hygiene has the jurisdiction to regulate all matters affecting health in the city and to perform all those functions and operations tha 
---


In [ ]:
chunked_embeddings = embedding_model.embed_documents(chunked_content)

In [ ]:
!gcloud storage cp gs://partner-genai-bucket/genai069/chunked_content.pkl .
!gcloud storage cp gs://partner-genai-bucket/genai069/chunked_embeddings.pkl .

Copying gs://partner-genai-bucket/genai069/chunked_content.pkl to file://./chunked_content.pkl
Copying gs://partner-genai-bucket/genai069/chunked_embeddings.pkl to file://./chunked_embeddings.pkl

Average throughput: 144.1MiB/s


In [ ]:
import pickle

chunked_content = pickle.load(open("chunked_content.pkl", "rb"))
chunked_embeddings = pickle.load(open("chunked_embeddings.pkl", "rb"))

In [ ]:
import google.cloud.logging
import logging

client = google.cloud.logging.Client()
client.setup_logging()

log_message = f"chunked contents are: {chunked_content[0][:20]}"
logging.info(log_message)

INFO:root:chunked contents are: The Health Code Thes


# Task 3 Prepare your vector database

In this section, you will set up a Firestore database to store the processed NYC Food Safety Manual chunks and their embeddings for efficient retrieval. You'll then build a search function to find relevant information based on a user query.

[Create a Firestore database](https://firebase.google.com/docs/firestore/manage-databases#create_a_database) with the default name of (default) in Native Mode and leave the other settings to default.

Next, in your Colab Enterprise Notebook populate a db variable with a Firestore Client.

Use a variable called collection to create a reference to a collection named food-safety.

Using a combination of your lists chunked_content and chunked_embeddings, add a document to your collection for each of your chunked documents. Each document can be assigned a random ID, but it should have a field called content to store the chunk text and a field called embedding to store a [Firestore Vector](https://firebase.google.com/docs/firestore/vector-search#write_operation_with_a_vector_embedding) of the associated embedding.

Create a vector index for your collection using your embedding field.

Note: A find_nearest() operation cannot be executed on a collection without an index. When attempted, the system will return an error message including instructions to create the index using a gcloud command.

Complete the function below to receive a query, get its embedding, and compile a context consisting of the text from the 5 documents with the most similar embeddings. This time, use the embed_query() method of the LangChain [VertexAIEmbeddings](https://python.langchain.com/v0.2/docs/integrations/text_embedding/google_vertex_ai_palm/#embed-single-texts) embedding_model to embed the user's query.

In [21]:
# Populate a db variable with a Firestore Client.
db = firestore.Client(project=PROJECT_ID)

# Use a variable called collection to create a reference to a collection named food-safety.
collection = db.collection('food-safety')

# Using a combination of our lists chunked_content and chunked_embeddings,
# add a document to your collection for each of your chunked documents.
for i, (content, embedding) in enumerate(zip(chunked_content, chunked_embeddings)):
    doc_ref = collection.document(f"doc_{i}")
    doc_ref.set({
        "content": content,
        "embedding": Vector(embedding)
    })

In [22]:
!gcloud firestore indexes composite create \
--collection-group=food-safety \
--query-scope=COLLECTION \
--field-config field-path=embedding,vector-config='{"dimension":"768", "flat": "{}"}' \
--project="qwiklabs-gcp-01-4db5c0a69bf4"

Create request issued
Created index [CICAgOjXh4EK].


In [ ]:
def search_vector_database(query: str):

  context = ""

  # 1. Generate the embedding of the query

  # 2. Get the 5 nearest neighbors from your collection.
  # Call the get() method on the result of your call to
  # find_nearest to retrieve document snapshots.

  # 3. Call to_dict() on each snapshot to load its data.
  # Combine the snapshots into a single string named context


  return context

In [23]:
def search_vector_database(query: str):
  context = ""
  query_embedding = embedding_model.embed_query(query)
  vector_query = collection.find_nearest(
    vector_field="embedding",
    query_vector=Vector(query_embedding),
    distance_measure=DistanceMeasure.EUCLIDEAN,
    limit=5,
  )
  docs = vector_query.stream()
  context = [result.to_dict()['content'] for result in docs]
  return context

Next, call the function with the query How should I store food? to confirm it's functionality.

In [24]:
search_vector_database("How should I store food?")

[' Store foods away from dripping condensate , at least six inches above the floor and with enough space between items to encourage air circulation. Freezer Storage Freezing is an excellent method for prolonging the shelf life of foods. By keeping foods frozen solid, the bacterial growth is minimal at best. However, if frozen foods are thawed and then refrozen, then harmful bacteria can reproduce to dangerous levels when thawed for the second time. In addition to that, the quality of the food is also affected. Never refreeze thawed foods, instead use them immediately. Keep the following rules in mind for freezer storage:  Use First In First Out method of stock rotation. All frozen foods should be frozen solid with temperature at 0°F or lower. Always use clean containers that are clearly labeled and marked, and have proper and secure lids. Allow adequate spacing between food containers to allow for proper air circulation. Never use the freezer for cooling hot foods. * * Tip: When receiv